In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from multiprocessing import Pool
import concurrent.futures
import time
from random import randint

In [13]:
#find number of pages in site to scrape

base_url = 'https://finance.sponser.co.il/finance/recommendation/index.php'
c = requests.get(base_url).content
soup=BeautifulSoup(c,"html.parser")
LastPage = int(soup.findAll("li",{"class":"c-pagination__item"})[-1].text)

BaseUrl = 'https://finance.sponser.co.il/finance/recommendation/index.php?page='
#LastPage = 40
allUrls = [BaseUrl+str(i) for i in range (1,LastPage+1)]

In [3]:
def scrape(url):
    #sleep(randint(0,3))
    c = requests.get(url).content
    soup=BeautifulSoup(c,"html.parser")
    table = soup.find("table",{"id":"portfolioTable"})
    a = pd.read_html(str(table))[0]
    return a

In [4]:
# the scraping code

# # parse all table with pandas read html 

# table = soup.find("table",{"id":"portfolioTable"})
# a = pd.read_html(str(table))[0]
# len(a)


# # parse row by row

# # allRows=soup.findAll("tr",{"class":"c-table__row u-border-top-zero"})

# # l = []
# # for tr in allRows:
# #     td = tr.find_all('span')
# #     row = [tr.text for tr in td]
# #     l.append(row)
# # b = pd.DataFrame(l,columns=['תאריך', 'מניה', 'סוג המלצה', 'בית השקעות', 'אנליסט', 'מחיר בעת המלצה',
# #        'מחיר יעד', 'מרחק מהיעד', 'מחיר אחרון'])

# simple scrape

In [5]:
%%time 


all_data = pd.DataFrame(columns = ['תאריך', 'מניה', 'סוג המלצה', 'בית השקעות', 'אנליסט', 'מחיר בעת המלצה',
       'מחיר יעד', 'מרחק מהיעד', 'מחיר אחרון'])

for i in range (1,LastPage+1):
    all_data = all_data.append(scrape(BaseUrl+str(i)),ignore_index=True)

Wall time: 3min


In [ ]:
# time for scraping 40 pages - 3 minutes
# lets try with multithreading
# with multithreading 40 pages ~ 50 seconds

# Multithreading

In [14]:
%%time 

#Multithreading
#---------------

with concurrent.futures.ThreadPoolExecutor(max_workers=min(LastPage,30)) as executor:
    data = list(executor.map(scrape, allUrls))    
    all_data_mt = pd.concat(data)

Wall time: 3min 53s


In [15]:
all_data_mt.head()

,תאריך,מניה,סוג המלצה,בית השקעות,אנליסט,מחיר בעת המלצה,מחיר יעד,מרחק מהיעד,מחיר אחרון
0,19/09/2021,אנרגיאן,קניה,בנק לאומי,אלה פריד,3384.0,4100.0,21.16%,3473.0
1,19/09/2021,אייאיאס,קניה,ווליו בייס,שי ליפמן,26260.0,38600.0,46.99%,27220.0
2,14/09/2021,פריים אנרגי,קניה,פרוסט אנד סאליבן,טיראן רוטמן,1439.0,1930.0,34.12%,1421.0
3,13/09/2021,אלטשולר שחם גמל,קניה,לאומי פרטנרס,מרב פישר שרוני,1829.0,2300.0,25.75%,1858.0
4,12/09/2021,אאורה,קניה,ווליו בייס,שי ליפמן,349.9,600.0,71.48%,394.2


In [16]:
all_data_mt.to_excel('israel_analysis.xlsx')